### <img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393/578 Máquinas de Aprendizaje - 2019-1 </h1>

<H3 align='center'> Tarea 3  </H3>
<hr style="height:2px;border:none"/>

**Temas**  
* Manipulaciones en numpy
* Arboles de clasificación y regresión
* Algunas Redes neuronales básicas
* Ensamblados, Adaboost y Random Forest
* Nociones de clases desbalanceadas


**Formalidades**  
* Equipos de trabajo de 2 personas (*Ambos estudiantes deben estar preparados para presentar la tarea el día de la entrega*)
* El entregable debe ser un _Jupyter Notebook_ incluyendo los códigos utilizados, los resultados, los gráficos realizados y comentarios. Debe seguir una estructura similar a un informe (se debe introducir los problemas a trabajar, presentar los resultados y discutirlos). Si lo prefiere puede entregar un _Jupyter Notebook_ por pregunta o uno por toda la tarea, con tal de que todos los entregables esten bien identificados y se encuentren en el mismo repositorio de _Github_.
* Se debe preparar una presentación del trabajo realizado y sus hallazgos. El presentador será elegido aleatoriamente y deberá apoyarse en el _Jupyter Notebook_ que entregarán. 
* Formato de entrega: envı́o de link del repositorio *privado* en _Github_, al correo electrónico del ayudante (*<alvaro.valderrama.13@sansano.usm.cl>*), en copia al profesor (*<cvalle@inf.utfsm.cl>*). Especificar el siguiente asunto: [INF393/578-2019 Tarea 3]. Invitar como colaborador al usuario de github "avalderr" para poder acceder al repositorio.
* Fecha de entrega y presentaciones: 13 de Septiembre. Hora límite de entrega: 23:50. Cualquier _commit_ luego de la hora límite no será evaluado. Se realizará descuento por atrasos en envío del mail. 

<hr style="height:2px;border:none"/>

La tarea se divide en tres partes:

[1.](#primero) Regresión para ubicación espacial  
[2.](#segundo) Clasificación con clases desbalanceadas    
[3.](#tercero) Clasificación en texto   

La tarea tiene ejemplos de códigos con los cuales pueden guiarse en gran parte, sin embargo solo son guias y pueden ser creativos al momento de resolver la tarea. Soluciones creativas o elegantes serán valoradas. También en algunas ocaciones se hacen elecciones arbitrarias, ustedes pueden realizar otras elecciones con tal de que haya una pequeña justificación de por qué su elección es mejor o equivalente.
Recuerden intercalar su código con comentarios y con celdas _Markdown_ con los comentarios de la pregunta y con cualquier analisis, fórmula (en $ \LaTeX $) o explicación que les parezca relevante para justificar sus procedimientos. 
Noten que en general cuando se les pide elegir algo o proponer algo no se evaluará mucho la elección en si, en cambio la argumentación detrás de la elección será lo más ponderado.
Si algun modelo se demora demasiado en correr en su maquina, no olvide que puede correr _Jupyter Notebooks_ en _Collab_ de Google, esto puede ser relevante para las maquinas más lentas al momento de realizar exploraciones con _K-folds_ por ejemplo.

<a id="primero"></a>
## 1. Regresión para ubicación espacial

Un problema que tiene aplicaciones para muchos servicios es el calculo de la ubicación espacial exacta de un dispositivo conectado dentro de un edificio o un complejo de edificios. Estando dentro de un aeropuerto nuestro celular nos podría dar indicaciones para llegar a nuestra puerta de embarque o dentro de un mall encontrar la tienda que encontramos serían aplicaciones que fácilmente se desprenden de la solución a este problema. Actualmente en exteriores el problema se soluciona fácilmente mediante el uso de GPS, sin embargo en interiores el problema se vuelve más complicado pues hay considerable pérdida de la señar GPS y muchas superficies de construcción pueden resultar reflectantes para algunos tipos de señales por lo cual el problema es mucho más dificil de modelar que el problema en el exterior. 


En esta primera parte de la tarea utilizaremos un _dataset_ creado en 2013 con más de 20000 mediciones de posición dentro de 4 edificios de la universidad de Jaume, disponible en https://archive.ics.uci.edu/ml/datasets/UJIIndoorLoc junto con una detallada descripción. Nuestra tarea será predecir la ubicación espacial en latitud, longitud, y altura (en pisos) de los distintos usuarios que realizaron esta tarea, a partir de las intensidades de señal medidas por los 520 distintos puntos de acceso inalambricos. Para esto, probaremos utilizaremos arboles de regresión, regresión lineal y redes neuronales, comparando los distintos desempeños y las ventajas y desventajas de cada aproximación. 


#### 1.a Carga de datos

Cargue los datos de entrenamiento y validación y júntelos en un solo DataFrame como muestra el código. 

De los datos entregados luego del _target_ solo consideraremos el tipo de celular utilizado, convirtiendolo a variables _dummies_ como muestra el código. 

Separe el _target_ del resto de los datos.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import scipy.stats as stats

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [1]:
from google.colab import files
uploaded = files.upload()

Saving trainingData.csv to trainingData.csv
Saving validationData.csv to validationData.csv


In [0]:
df_tr = pd.read_csv("trainingData.csv")
df_val = pd.read_csv("validationData.csv")

In [8]:
df_tr.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,WAP013,WAP014,WAP015,WAP016,WAP017,WAP018,WAP019,WAP020,WAP021,WAP022,WAP023,WAP024,WAP025,WAP026,WAP027,WAP028,WAP029,WAP030,WAP031,WAP032,WAP033,WAP034,WAP035,WAP036,WAP037,WAP038,WAP039,WAP040,...,WAP490,WAP491,WAP492,WAP493,WAP494,WAP495,WAP496,WAP497,WAP498,WAP499,WAP500,WAP501,WAP502,WAP503,WAP504,WAP505,WAP506,WAP507,WAP508,WAP509,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-92,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710


In [10]:
print(df_tr.columns)

Index(['WAP001', 'WAP002', 'WAP003', 'WAP004', 'WAP005', 'WAP006', 'WAP007',
       'WAP008', 'WAP009', 'WAP010',
       ...
       'WAP520', 'LONGITUDE', 'LATITUDE', 'FLOOR', 'BUILDINGID', 'SPACEID',
       'RELATIVEPOSITION', 'USERID', 'PHONEID', 'TIMESTAMP'],
      dtype='object', length=529)


In [0]:
df_tr = pd.read_csv("UJIndoorLoc/trainingData.csv")
df_val = pd.read_csv("UJIndoorLoc/validationData.csv")

df = shuffle(df_tr.append(df_val))

target_labels = [520,521,253]

target_col =  df.columns[[520,521,253]] 
attribute_col = # . . . 

x_un = pd.get_dummies(df.PHONEID)
categorical_columns = x_un.columns
x_un[attribute_col] = df[attribute_col]

y_u = # . . .

Separe los datos a partir de su índice, dejando un 80% de datos en el _train set_ y un 20% de validación. 
¿Por qué no es mala idea en este caso separar los datos a partir de su índice? ¿Qué problema podría tenerse en caso de no haber hecho `shuffle` al momento de cargar los datos? 


In [0]:
x_tr_un = x_un.iloc[0:16838]
# . . .

#### 1.b Estandarización de los datos

Utilice _Standar Scaler_ para los atributos no categóricos y para los datos categóricos simplemente sustraiga 0.5, como muestra el código. ¿Tiene realmente sentido estandarizar los datos categóricos (binarios) o le parece mejor la aproximación elegida?

Estandarice igualmente el _target_. Cabe mencionar que la tercera componente del target es el piso en el que se encuentra, variable categórica, preferiremos utilizarla como variable continua para ver que tan bien nuestros algoritmos pueden aproximar la ubicación y para facilitar la comparación de distintos modelos. 

In [0]:
# . . . 
x_tr = pd.np.concatenate((scaler_x.transform(x_tr_un[attribute_col]), x_tr_un[categorical_columns].values-0.5), axis=1)
# . . . 


#### 1.c Primer Arbol
Entrenaremos ahora un árbol de regresión con produndidad 3. Como métrica utilizaremos el Error Absoluto Medio o MAE a lo largo del resto de esta pregunta. ¿Qué tan bueno le parece el _score_ (coeficiente $R^2$) obtenido por el regresor?

Compare el desempeño del árbol, tanto en MAE como en _score_, en validación y entrenamiento, con respecto a un regresor lineal tradicional. ¿Cómo se comparan los tiempos de entrenamiento? ¿Le parece justificado el costo adicional?

In [0]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(criterion='mae',
                            max_depth=3
                            )
# . . . 

#### 1.d _Stump_
Entrene ahora un _Stump_ o arbol de profundidad 1. ¿Cómo cambia el error (MAE) en entrenamiento y validación con respecto al árbol de la pregunta anterior? Discuta sobre el numero de parámetros entrenables que tendrá cada modelo.

In [0]:
stump =  DecisionTreeRegressor(criterion='mae',
                            max_depth=1
                            )
# . . . 

#### 1.e Profundidad máxima
Explore distintos valores de la profundidad máxima del árbol. Para esto, realice K-fold sobre al menos 10 valores distintos de la profundidad del árbol y compare el promedio de los MAE de validación. También guarde los promedios de MAE de _train_ y grafique como estos varían a medida varia la profundidad máxima. Idealmente debería considerar un rango entre profundidad máxima 1 y 20. Note que este paso puede resultar largo en tiempo de cómputo, podría ser recomendable correrlo en _Collab_ en caso de tener máquinas más lentas. 

Finalmente, entrene el arbol con profundidad óptima sobre todos los datos de entrenamiento y calcule su MAE sobre validación. ¿Mejoramos el desempeño obtenido anteriormente con los árboles sin sintonizar? 

#### 1.f Random Forest

En la pregunta anterior vimos que una profundidad de árbol muy grande puede llevarnos a _overffiting_, pues la cantidad de "decisiones" que puede aprender el modelo aumentan significativamente y el modelo tenderá a aprender de memoria el conjunto de entrenamiento, perdiendo capacidad de generalización. Otra aproximación para evitar _overffiting_ no utilizar todos los datos para entrenar, estrategia implementada por _Random Forest_. 

Discuta por qué el utilizar muchos modelos que no conocen todos los datos puede ayudar a mejorar la capacidad de generalización, evitando así que el modelo final este sobreajustado. Entrene luego un _Random Forest Regressor_, con a lo menos 10 árboles. Discuta sobre que elección realizar sobre la profundidad máxima de estos árboles. ¿Cómo se compara el desempeño de _Random Forest_ frente a un árbol por si solo?

In [0]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=20,
                               criterion='mae',
                               max_depth=5
    )

#### 1.g Otros parámetros

Por último, investigue un poco sobre los otros parámetros de los árboles y sus implicancias. Luego, entrene un arbol modificando algunos de sus parámetros de una forma que piensen podría haber mejorado el desempeño. Discuta

In [0]:
tree = DecisionTreeRegressor(criterion='mae',
                            max_depth=None,
                            min_impurity_decrease=0.1, # for instance
                            min_samples_split=int(x_tr.shape[0]*0.05) #for instance
                            )

<a id="segundo"></a>
## 2. Clasificación con clases desbalanceadas

En las tareas de clasificación supervisada, buscamos que mediante los ejemplos etiquetados la maquina pueda aprender los atributos inerentes a las distintas clases. Sin embargo, en muchos casos reales de clasificación, la cantidad de ejemplos de cada clase son muy dispares, en cuyo caso hablamos de clases desbalanceadas. Si uno no aplica estrategias para compensar este problema, la maquina aprenderá muy bien las caracteristicas de la clase más representada pero no logrará extraer información generalizable de la clase menos representada. Además, debemos tener cuidado al momento de interpretar las distintas métricas a las cuales tenemos acceso, pues debemos tomar en cuenta el balance de las clases y la naturaleza del problema para evaluar realmente que tan bueno es el desempeño. 



Para esta parte de la tarea utilizaremos el siguiente _dataset_ publicado en Kaggle: https://www.kaggle.com/mlg-ulb/creditcardfraud. Este conjunto de datos contiene unos 285000 ejemplos de transacciones con tarjetas de crédito reales, realizadas en 2013 por clientes europeos. Los datos son totalmente anónimos y son el resultado de un PCA a partir de los datos originales excepto por las columnas `Time` con el tiempo en segundos desde la primera transacción, `Amount` con el monto de la transacción y `Class` que indica si la transacción es fraudulenta o no. La tarea en cuestión consiste en lograr predecir cuando una transacción es fraudulenta o no automaticamente, para así poder detenerla antes de que se termine. Sin embargo, por la naturaleza de las transacciones bancarias, este problema es desbalanceado, de hecho de los 285000 transacciones registradas, solo 492 son fraudulentas! A lo largo de esta pregunta pondremos en evidencia los problemas que se originan de este desbalance de clases y trataremos de dar luces a algunas herramientas que nos permitan sortear esos problemas y como medir realmente el desempeño en esta clase de problemas. 

#### 2.a Carga de datos
Descargue los datos y cárguelos usando pandas. Haga una exploración rápida de los datos, cuantos datos hay, como se reparten sus valores.

Grafique la matriz de correlación, a primera vista parece relevante mantener la columna `Time`? En su opinión, conociendo la naturaleza del problema le parece relevante esa información para predecir el _target_?

Cuantos ejemplos hay de cada clase? Son las clases desbalanceadas efectivamente? 

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('creditcard.csv')

df.describe()

# . . . 

plt.imshow(df.corr())

# . . .

df.loc[lambda x: x.Class != 0].shape

#### 2.b Ligero preprocesamiento

* Escale la columna `Amount`. Por qué es aconsejable realizar esto? Le parece necesario realizarlo en el resto de las columnas? 

* Separe los atributos del _target_.

* Separe luego los datos en _Training set_ y _Validation set_, con un 20% de los datos como validación y el resto como _train_. Asegurese que se mantienen las proporciones de ejemplos de cada clase en ambos sets. 

* Qué ocurriría si por ejemplo todos los ejemplos de la clase 1 quedaran en el _validation set_, que haría cualquier máquina de aprendizaje al aprender solo con la clase 0?

In [0]:
from sklearn.preprocessing import StandardScaler

# . . .

df['Amount'] = scaler.transform(df['Amount'].values.reshape(-1,1))

x = df[df.columns[1:-2]]
y = df.Class

from sklearn.model_selection import train_test_split

#### 2.c Regresor logístico

Entrene un regresor logístico con los datos obtenidos de la pregunta anterior y calcule su desempeño (_score_ por ahora) sobre los datos de validación. A priori le parece un buen desempeño? 

In [0]:
from sklearn.linear_model import LogisticRegression
# . . . 
# . . . 

#### 2.d _Always_ 0

Suponga un modelo le entrega como resultado en el conjunto de validación el siguiente `y_pred`. ¿Qué está haciendo este predictor? ¿Qué _score_ obtendría tal modelo sobre el conjunto de validación? ¿Le parece que el modelo tenga un buen desempeño? Concluya sobre la calidad del _accuracy_ que obtenemos con `.score` para evaluar el desempeño de estos modelos. 

In [0]:
y_pred = pd.np.zeros(y_val.shape)

# . . .

#### 2.e Matriz de confusión

Investigue un poco sobre los valores que conforman la matriz de confusión o _confusion matrix_. Comente sobre la significancia de los distintos valores para el problema en cuestión. 

Escriba una función que a partir de un modelo o de los valores predecidos por un modelo, grafique la matriz de confusión. 

In [0]:
# Puede usar libreria seaborn para realizar facilmente heatmaps anotados: 
import seaborn as sns
# . . .
sns.heatmap( confusion_matrix(y_test, model.predict(X_test))/len(X_test),cmap='winter', annot=True)    
# . . .

Grafique la matriz de confusión para la regresión logística realizada en el punto c y para los valores `y_pred` del punto d. Le parecen parecidos ahora ambos modelos? 

#### 2.f Curva ROC
La curva ROC tiene bastante utilidad para evaluar la calidad de distintos modelos en estos casos. Investigue un poco sobre su significado para poder interpretarla correctamente y luego escriba una función que reciba un modelo entrenado y la grafíque. 

Note que `sklearn` trae implementado una función que entrega los puntos de la curva ROC, note eso sí que debe entregarle las probabilidades usando el método `predict_proba` del modelo entrenado. 

Para evaluar distintos modelos utilizará bastante las curvas ROC y la matriz de confusión, por lo que sería recomendable escribir una función que reciba un modelo y datos y realice tanto la curva ROC como la matriz de confusión, aunque escribir tal función no es requisito. 

In [0]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_ts, model.predict_proba(x_ts)[:,1],pos_label=1)

plt.plot(fpr, tpr, label=# . . . )

Utilice la función definida para su modelo de regresión logística. Comente el gráfico obtenido. 

#### 2.g Arbol de Clasificación
Entrene un arbol de clasificación para tratar de resolver el problema. Pruebe unas cuantas profundidades máximas distintas y elija una. Si quiere puede modificar los otros hiperparámetros del modelo. 

Evalúe su desempeño respecto a los modelos lineales usando las métricas relevantes y compare su costo computacional. Discuta.

In [0]:
tree = DecisionTreeClassifier(
            criterion='gini',
            max_depth=3
)
# . . .

#### 2.h _Undersampling_

Considerando que la gran mayoría del aprendizaje se realiza en los ejemplos no fraudulentos, una aproximación para tratar de mejorar el desempeño de los modelos entrenados es simplemente reducir la cantidad de ejemplos de una clase para obtener nuevos conjuntos de datos con clases más balanceadas. Así, el aprendizaje se realiza de manera más balanceada entre ambas clases y se logran extraer mejor los atributos de cada clase. 

Complete la siguiente función para que realice un _undersampling_ de la clase 0 para obtener una proporción en el nuevo conjundo de datos de `times` veces el numero de transacciones normales frente a fraudulentas. Comente sobre la decisión del parámetro `replace` de la función `numpy.random.choice`. 


Note que la función recibe el _DataFrame_ con todos los datos. Se pide que retorne los conjuntos de _train_ y _validation_ para luego poder entrenar un modelo. 

In [0]:
def undersample(df, times = 10, validation_ratio = 0.2):
    fraud_indices = pd.np.array(df[df.Class==1].index)
    normal_indices = # . . .

    Count_Fraud_transacation = # . . .
    
    Normal_indices_undersample = pd.np.array(
    pd.np.random.choice(
        # wich indices . . . ,
        Count_Fraud_transaction * #something,
        replace = # True or False?)
    )
    
    undersample_data = pd.np.concatenate([fraud_indices,Normal_indices_undersample])
    undersample_data = df.iloc[undersample_data]
    
    # . . . X, y
    # train test split
    return x_train, y_train, x_test, y_test

#### 2.i Entrenar con _Undersampling_

Pruebe distintos valores para el parámetro `times` y evalue si hay diferencias significativas en la curva ROC y la matriz de confución para el mismo árbol de clasificación entrenado anteriormente. Evidentemente el número de ejemplos cambia a medida cambia el parámetro `times`, por lo que la matriz de confusión debe dividirla por la cantidad de datos considerados para realizar comparaciones justas, de proporciones de ejemplos clasificados y no de número de ejemplos. 

¿Como se comparan estos modelos con el modelo con todos los datos (recuerde que las comparaciones deben hacerse calculando el desempeño sobre el conjunto de validación completo con el modelo una vez entrenado)? ¿Ve alguna ventaja en el uso de _undersampling_ frente al uso de todos los datos? 

#### 2.j _Oversampling_

Otra aproximación para resolver el problema de las clases desbalanceadas es hacer _oversampling_. A partir de una muestra relativamente pequeña de ejemplos, en este caso de los casos de fraude, se busca crear un conjunto de datos más grande similar al inicial, que permita al modelo aprender las caracteristicas de esa clase, buscando nuevamente obtener clases más balanceadas que originalmente, pero esta vez sin reducir la cantidad de ejemplos. Esta aproximación tiene la ventaja que no reduce la cantidad de ejemplos pero trae la complicación de decidir cómo se van a crear los datos nuevos.

Dos aproximaciones bastante estandar son SMOTE y ADASYN. Investigue un poco que hace SMOTE y que diferencia implemente ADASYN. Por qué cree puede ser deseable buscar que los datos sintéticos no sean linealmente dependientes de los datos originales? 

Elija alguno de los dos métodos e impelementelo con el árbol entrenado previamente. Comente sobre el desempeño de tal modelo. 

In [0]:
!pip install  imbalanced-learn

from imblearn.over_sampling import SMOTE, ADASYN
ada = ADASYN()
# . . .
x_tr_r, y_tr_r  = smo.fit_resample(x_tr,y_tr)


Note que tanto SMOTE como ADASYN tienen muchos parámetros que podrían afectar la calidad de los datos generados. No se requiere que elijan tales parametros (los por defecto bastan), pero se valorará un pequeño comentario sobre los parámetros de los métodos y qué representan en la heuristica utilizada. 

#### 2.k Redes neuronales _Feed Forward_ 

Por útimo, probaremos utilizar una red neuronal densa sobre los datos aumentados. Las Redes Neuronales son una herramienta muy poderosa pero comparativamente costosa frente a otros modelos. Entrenar una red neuronal puede resultar muy costos computacionalmente, por su gran numero de parámetros entrenables y por el caracter iterativo de su entrenamiento. Además, el gran numero de parámetros hace que las redes neuronales sean particularmente sensibles al _overfitting_, por lo cual suele ser necesario tener grandes cantidades de datos para poder entrenar una red neuronal obteniendo desempeños que justifiquen su costo adicional.

Por esto, entrenaremos la red neuronal sobre los datos aumentados. Básese en el código siguiente para crea la red neuronal. Explique los parámetros que se pasan en cada etapa de su creación, para esto puede leer la documentación de keras. Finalente, entrene "completamente" la red, es decir hasta que se estanque el error de validación o comience a aumentar. ¿Cómo se compara la red frente a otros modelos, tanto en desempeño como en costo? 

In [0]:
from keras.layers import Dense
from keras.models import Sequential

model = Sequential()

model.add(Dense(256,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='sgd',
             loss='binary_crossentropy',
             metrics=['accuracy'])
history = model.fit(x_tr.values,y_tr.values, epochs=20, validation_data=(X_test,y_test))

# note that history.history gives you the values of tr and val errors and accuracies

<a id="tercero"></a>
## 3. Clasificación en texto

Un problema muy complejo, en el cual actualmente se esta avanzando a pasos de gigantes en comparación con los avances de hace unos cuantos años, es en la comprehensión de texto, o de lenguaje natural en general. Nuestra manera de construir significado a partir de las palabras es mucho más compleja de lo que nos damos cuenta y supone un gran grado de conocimiento previo que hemos adquirido a lo largo de nuestras vidas. Lograr que una maquina de aprendizaje pueda extraer significado o interpretaciones a partir de un texto es un problema bastante actual, en el cual aún se hacen avances día a día. En esta pregunta utilizaremos una base de datos de texto, extraida de _reviews_ de peliculas en la página IMDB. Cada uno de estos _reviews_ esta etiquetado como positivo o negativo. Nuestra tarea será predecir esta etiqueta a partir de la secuencia de palabras encodeadas como frecuencias de cada ejemplo.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/New-imdb-logo.png/1024px-New-imdb-logo.png"  style="height:6cm;"  />

Para esto probaremos distintas técnicas, incluyendo Perceptron, Redes densas _feed forward_ y ensamblados. Si bien ninguna de estas aproximaciones son las usualmente utilizadas en la actualidad para procesamiento de texto (_words embeddings_, redes recurrentes y redes convolucionales, entre muchos otros), esta pregunta servirá como una buena aproximación a las problematicas usualmente encontradas en estas técnicas y al momento de trabajar con texto. 


#### 3.a Carga de datos y preprocesamientos

Cargue los datos. Utilizaremos 45000 de los datos como entrenamiento y el resto como validación. Investigue sobre la base de datos en cuestión y la representación de los datos cargados: ¿qué representa cada entrada de `x_tr`?

Comente ligeramente sobre los parámetros `num_words` y `skip_top`. ¿Por qué podría ser deseable eliminar las palabras más comunes?

Utilize luego la función `vectorize_sequences` para transformar los inputs del modelo. ¿Qué realiza esta función? ¿Qué valor debería tener el parámetro `dimension` en este caso?  ¿Qué información perdemos y qué información preservamos en esta transformación?

In [0]:
import numpy as np
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


num_words = 500
(x_tr, y_tr), (x_val, y_val) = imdb.load_data(num_words=num_words, skip_top=5)

def vectorize_sequences(sequences, dimension= # fill it yourself ):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_tr = vectorize_sequences( # . . .)

#### 3.b Perceptrón **(Opcional)**
Entrene un perceptrón con activación sigmoidea, optimizador _Stochastic Gradient Descent_, por a lo menos 15 _epochs_, como muestra el código siguiente. 

Discuta sobre la dimensión de `x_tr` y la cantidad de parámetros del modelo deacuerdo al método `.summary` del modelo. 

Grafique el error o el _accuracy_ segun su preferencia, tanto sobre entrenamiento como validación, a medida pasan los _epochs_. Note esta información puede obtenerla desde `history.history`.

In [0]:
per = Sequential()
per.add(Dense(1,activation='sigmoid'))
per.compile('sgd',loss='binary_crossentropy',metrics=['accuracy'])
history = per.fit(x_tr,y_tr, epochs=15, validation_data=(x_val,y_val))
per.summary()

#### 3.c Red con una capa oculta
Ahora, haremos nuevamente un modelo secuencial, al cual le agregarémos una capa oculta con activación ReLu. ¿Qué realiza la función ReLu (puede ejemplificar con gráficos)?

Entrene esta red nuevamente con los mismos parámetros que la red anterior. ¿Mejoran los desempeños? ¿Cuál es el número de parámetros de este modelo? Grafíque nuevamente como varía la métrica a medida pasan las _epochs_. ¿Alcanza a notar sobreajuste?

Cambie la función de activación por otra de su elección (e.g. $tanh$ o activación lineal). ¿Cambia el desempeño? 

Compare finalmente el desempeño con una regresión lineal. ¿Cómo hace la red para obtener representaciones no-lineales que la regresión lineal no puede obtener? ¿Por qué se podría argumentar que una red neuronal densa _feed forward_ tiene un sabor a ensamblado de regresiones lineales en un principio? 

In [0]:
model = # . . .
model.add(Dense(64, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
# . . . 

from sklearn.linear_model import LinearRegression
# . . .

#### 3.d Exploración de número de neuronas ocultas

Utilizando la misma estructura de la red entrenada en la pregunta anterior, evalue como varía el _accuracy_ sobre validación y entrenamiento a medida se cambia el numero de neuronas. Explore numeros de neuronas en potencias de 2 entre 2 y 256. En cada iteración debe considerar el mejor _accuracy_ sobre validación y el _accuracy_ de entrenamiento de ese mísmo _epoch_, pues ese sería el modelo que se preferiría. 

Grafique como variaron los _accuracy_ a medida cambió el número de neuronas y discuta. 

In [0]:
for i in range(1,9):
    # do model
    model.add(Dense(units=2**i), activation # . . .)
    # compile and train model
    # you can user history.history to get best accuracy and save it
    

#### 3.e Pesos de la red

Obtenga los pesos de la red entrenada a completitud anteriormente usando el método `.get_weights` y grafíque un historgrama. Note que para cada capa se obtienen por separado los pesos de los datos y del intercepto. Así en el índice 0 obtenemos los pesos de los datos de entrada, en el 1 los pesos del intercepto de entrada, en el 2 los pesos de la capa oculta y en el 3 los pesos del intercepto de la capa oculta. Solo se requiere graficar el primero.

Compare luego ese histograma con el obtenido al graficar los pesos de una red sin entrenar, como muestra el código. Discuta lo que observa. 

In [0]:
untrained = Sequential()
untrained.add(Dense(32, activation='relu'))
untrained.add(Dense(1,activation='sigmoid'))
untrained.compile(SGD(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])
untrained.fit(x_tr,y_tr,epochs=0)  # the easy way to create the weights without training, though there are more purists ways

plt.hist(untrained.get_weights()[0])
plt.show 
# . . . 

#### 3.f Extreme learning Machine **(Opcional)**
En esta pregunta implementaremos una _extreme learning machine_ o ELM. Este modelo puede asimilarse a una red neuronal como la entrenada anteriormente pero donde los pesos que conectan al _input_ y la capa oculta no son entrenables. Podemos realizar esto manualmente en algunos casos o utilizando igualmente keras como muestra el código. 
Entrene una ELM y compare los tiempos de entrenamiento y desempeños obtenidos para redes con el mismo número de neuronas. Compare tambien el número de parámetros entrenables respecto redes densas del mismo tamaño. 

Pruebe un par de tamaños de red distintos, considerando la cantidad de parámetros libres de los modelos en cuestión y comente. 

In [0]:
ELM = Sequential()
ELM.add(Dense(512, activation='relu',trainable=False))
ELM.add(# output layer )
# . . .

#### 3.g Autoencoders  **(Opcional)**
Otra aproximación muy ingeniosa para mejorar el desempeño de algunos modelos es el uso de _autoenconders_. Comente brevemente sobre qué realizar un _autoencoder_ y sobre por qué no necesita conocer el target para entrenarse.

Esta característica de los _autoencoders_ han permitido mejorar drásticamente muchas aplicaciones reales donde se tienen enormes cantidades de datos sin _target_ y menos datos ya etiquetados (por ejemplo, en reconocimiento de imagenes google tiene acceso a una cantidad en la práctica ilimitada de fotos de sus usuarios pero muy pocas fotos "etiquetadas" en comparación), aprendiendo representaciones relevantes de los datos y luego transfiriendo ese conocimiento. En esta pregunta utilizaremos un autoencoder para tratar de mejorar el desempeño de la red neuronal densa. 

Para esto primero entrenaremos una red neuronal densa que tiene como _input_ y _target_ nuestros `x_tr`. Una vez entrenada esa red, extraeremos sus pesos de la capa escondida para obtener nuestra capa _encoder_ y nuestra capa _decoder_. Finalmente, utilizaremos nuestra capa de _encoding_ como la primera capa de un nuevo modelo (más pequeño), el cual luego entrenaremos sin modificar los pesos de la capa _encoding_.

Note que este procedimiento también puede hacerse como un preprocesamiento de los datos, sin embargo les propongo esta manera pues pone en evidencia el proceso de _transfer learning_ que comentabamos anteriormente. 

In [0]:
autoencoder = Sequential()
autoencoder.add(Dense(512, activation='relu')) # you can change the number of units
autoencoder.add(Dense(x_tr.shape[1]))
# compile with binary_crossentropy and fit (if the representation wasn't binary it might be a better idea to use mse)
    
# get the layers
encoder = autoencoder.layers[0]
decoder = autoencoder.layers[1]

#create the new model
model = Sequential()
model.add(encoder)
model.layers[0].trainable = False  # we dont want to train the encoder
# add the hidden and output layers

# compile and fit

¿Cuál es el factor de compresión (la representación que induce el autoencoder frente a la representación original)? ¿Cómo considera los resultados tomando en cuenta que en la practica el nuevo modelo aprende con muchos menos atributos, si bien estos atributos fueron entrenados por otro modelo? 

#### 3.h Bagging **(Opcional)**

Crearemos ahora un ensamblado utilizando la técnica de _Bagging_, ensamblando las predicciones de distintos árboles para predecir el resultado. 

Explique brevemente el procedimiento realizado por _Bagging_, en particular por qué los distintos arboles no se entrenan idénticamente y el significado de los parámetros que aparecen en el código. 

In [0]:
from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                        n_estimators=100,
                        max_samples=0.5,
                        max_features=0.7
                       )
# fit and compare 

#### 3.i AdaBoost
_Adaptative Boosting_ o AdaBoost es otro meta-algoritmo que crea ensamblados a partir de modelos más simple. Entrene un modelo de AdaBoost con estimador "pequeño", por ejemplo puede usar un arbol, una regresión lineal u otros modelos de `sklearn` y un límite de estimadores (`n_estimators`) de a lo menos 200. 

Calcule su _accuracy_ y comparelo con los demás métodos. 

In [0]:
from sklearn.ensemble import AdaBoostClassifier
# . . . 

Discuta sobre el modelo que encontró el mejor `accuracy`. ¿Por qué cree que esta representación le acomoda a la red neuronal y no a los arboles de desición? 